Used the pre-trained glove embedding model in this trial.
Add Convolution layers

In [1]:
import read_data 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, TensorBoard, CSVLogger
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from pandas.io.pickle import to_pickle, read_pickle
save_dir = 'text_classifier3/'
import os
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

Using TensorFlow backend.


In [2]:
#%% imbeding vectors
#download
glove_dir = 'glove_data/'
if not os.path.exists(glove_dir):
    os.mkdir(glove_dir)
if not os.path.exists(glove_dir+'glove.6B.zip'):
    import subprocess
    subprocess.run(["wget",'-P'+glove_dir, "http://nlp.stanford.edu/data/glove.6B.zip"])
    print('downloaded')
if not os.path.exists(glove_dir+'glove.6B.50d.txt'):
    import zipfile
    zip_ref = zipfile.ZipFile(glove_dir+'glove.6B.zip', 'r')
    zip_ref.extractall(glove_dir)
    zip_ref.close()
    print('unziped')

In [3]:
embeddings_index = {}
with open(glove_dir+'glove.6B.50d.txt') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [4]:
#%% first steps
books,genre,excerpt = read_data.read_text_data()
id_train,id_val = read_data.read_ids()

MAX_WORDS = 8000
tokenizer = Tokenizer(num_words=MAX_WORDS)
tokenizer.fit_on_texts(excerpt.values())
sequences = tokenizer.texts_to_sequences(excerpt.values())

word_index = tokenizer.word_index
to_pickle(tokenizer, save_dir+'text_tokenizer.pickle')
print(len(word_index))


169991


(0, 10000)

In [5]:
lens = [len(s) for s in sequences]
sns.distplot(lens)
plt.xlim((0,10000))

In [6]:
MAXLEN = 4000
data = pad_sequences(sequences, maxlen=MAXLEN)

#labels = to_categorical(np.asarray(labels))
labels = ['Fiction' in genre[id] for id in excerpt.keys()]
labels = np.array(labels).reshape((-1,1))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (4166, 4000)
Shape of label tensor: (4166, 1)


In [7]:
# split the data into a training set and a validation set

train_index = [id in id_train for id in excerpt.keys()]
val_index = [id in id_val for id in excerpt.keys()]
x_train = data[train_index]
y_train = labels[train_index]
x_val = data[val_index]
y_val = labels[val_index]
print(x_train.shape,y_train.shape,x_val.shape,y_val.shape)

to_pickle((x_train, y_train, x_val, y_val),save_dir+'text_dataset.pc')

(2922, 4000) (2922, 1) (1244, 4000) (1244, 1)


In [8]:
# prepare embedding layer
embedding_vecor_length = 50 # this is determined by the glove data!!!
num_words = min(MAX_WORDS, len(word_index))
embedding_matrix = np.zeros((num_words, embedding_vecor_length))
for word, i in word_index.items():
    if i >= MAX_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(num_words,
                            embedding_vecor_length,
                            weights=[embedding_matrix],
                            input_length=MAXLEN,
                            trainable=False)

In [11]:

model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(50,dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(1, activation='tanh'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

checkpoint = ModelCheckpoint(save_dir+"text_best_model.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
#early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')
with open(save_dir+'text_csvlogger.csv','w') as f:
    f.write('')
csvlog = CSVLogger(save_dir+'text_csvlogger.csv',append=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4000, 50)          400000    
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 3998, 32)          4832      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 1999, 32)          0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 100)               33200     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 438,133
Trainable params: 38,133
Non-trainable params: 400,000
_________________________________________________________________
None


In [12]:
model.fit(x_train, y_train, epochs=5, batch_size=64,
          callbacks = [checkpoint, csvlog],
          validation_data=(x_val, y_val))

#scores = model.evaluate(x_val, y_val, verbose=0)
#print("Accuracy: %f%%" % (scores[1]*100))

model.save(save_dir+'text_last_model.h5')
print(checkpoint.best)

Train on 2922 samples, validate on 1244 samples
Epoch 1/5
2922/2922 [==============================] - 176s - loss: 0.8455 - acc: 0.4976 - val_loss: 0.6485 - val_acc: 0.6608
Epoch 2/5
2922/2922 [==============================] - 172s - loss: 0.6652 - acc: 0.6331 - val_loss: 0.6378 - val_acc: 0.6543
Epoch 3/5
2922/2922 [==============================] - 173s - loss: 0.6334 - acc: 0.6362 - val_loss: 0.6136 - val_acc: 0.6624
Epoch 4/5
2922/2922 [==============================] - 173s - loss: 0.6244 - acc: 0.6550 - val_loss: 0.5857 - val_acc: 0.6768
Epoch 5/5
2922/2922 [==============================] - 172s - loss: 0.6379 - acc: 0.6259 - val_loss: 0.7355 - val_acc: 0.4574
0.676848874215


In [14]:
#%% continue training

#model = load_model(save_dir+'text_last_model.h5')
#x_train, y_train, x_val, y_val = read_pickle(save_dir+'text_dataset.pc')

model.fit(x_train, y_train, epochs=5, batch_size=64,
          callbacks = [checkpoint, csvlog],
          validation_data=(x_val, y_val))

model.save(save_dir+'text_last_model.h5')
print(checkpoint.best)

Train on 2922 samples, validate on 1244 samples
Epoch 1/5
2922/2922 [==============================] - 172s - loss: 0.6247 - acc: 0.6571 - val_loss: 0.5489 - val_acc: 0.6865
Epoch 2/5
2922/2922 [==============================] - 171s - loss: 0.5779 - acc: 0.6999 - val_loss: 0.4973 - val_acc: 0.7629
Epoch 3/5
2922/2922 [==============================] - 191s - loss: 0.5635 - acc: 0.7214 - val_loss: 0.5112 - val_acc: 0.7235
Epoch 4/5
2922/2922 [==============================] - 319s - loss: 0.5612 - acc: 0.7245 - val_loss: 0.4794 - val_acc: 0.7355
Epoch 5/5
2922/2922 [==============================] - 327s - loss: 0.5695 - acc: 0.7163 - val_loss: 0.5408 - val_acc: 0.7629
0.762861736909


In [15]:
model.fit(x_train, y_train, epochs=5, batch_size=64,
          callbacks = [checkpoint, csvlog],
          validation_data=(x_val, y_val))

model.save(save_dir+'text_last_model.h5')
print(checkpoint.best)

Train on 2922 samples, validate on 1244 samples
Epoch 1/5
2922/2922 [==============================] - 169s - loss: 0.5488 - acc: 0.7385 - val_loss: 0.5156 - val_acc: 0.7444
Epoch 2/5
2922/2922 [==============================] - 172s - loss: 0.5299 - acc: 0.7543 - val_loss: 0.4523 - val_acc: 0.7637
Epoch 3/5
2922/2922 [==============================] - 170s - loss: 0.5636 - acc: 0.7337 - val_loss: 0.5652 - val_acc: 0.7283
Epoch 4/5
2922/2922 [==============================] - 169s - loss: 0.5296 - acc: 0.7550 - val_loss: 0.4583 - val_acc: 0.8006
Epoch 5/5
2922/2922 [==============================] - 206s - loss: 0.4939 - acc: 0.7738 - val_loss: 0.4542 - val_acc: 0.7814
0.800643086817


In [ ]:
model.fit(x_train, y_train, epochs=5, batch_size=64,
          callbacks = [checkpoint, csvlog],
          validation_data=(x_val, y_val))

model.save(save_dir+'text_last_model.h5')
print(checkpoint.best)